In [1]:
import pandas as pd
import geopandas as gpd
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [2]:
df_tv = pd.read_csv('../artifacts/data/cleaned/television.csv', on_bad_lines='skip')
df_radio = pd.read_csv('../artifacts/data/cleaned/radio.csv', on_bad_lines='skip')
df_newspaper = pd.read_csv('../artifacts/data/cleaned/newspaper.csv', on_bad_lines='skip')
df_population = pd.read_csv('../artifacts/data/cleaned/population_groups.csv')
df_newsfacts = pd.read_csv('../artifacts/data/cleaned/pew_newsfacts.csv', on_bad_lines='skip')
df_county = pd.read_csv('../artifacts/shapefiles/county.csv')
df_shapefile_county = gpd.read_file('../artifacts/shapefiles/cb_2018_us_county_500k.shp')

In [18]:
df_shapefile_county 


,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."
...,...,...,...,...,...,...,...,...,...,...
3228,31,073,00835858,0500000US31073,31073,Gosper,06,1186616237,11831826,"POLYGON ((-100.09510 40.43866, -100.08937 40.4..."
3229,39,075,01074050,0500000US39075,39075,Holmes,06,1094405866,3695230,"POLYGON ((-82.22066 40.66758, -82.19327 40.667..."
3230,48,171,01383871,0500000US48171,48171,Gillespie,06,2740719114,9012764,"POLYGON ((-99.30400 30.49983, -99.28234 30.499..."
3231,55,079,01581100,0500000US55079,55079,Milwaukee,06,625440563,2455383635,"POLYGON ((-88.06959 42.86726, -88.06959 42.872..."


In [3]:
def process_media_sales_volume(df_media):
    """
    Converts 'sales_volume' to numeric, removes entries with missing 'sales_volume',
    and filters entries with 'sales_volume' below a specified threshold.

    Parameters:
        df_media (pd.DataFrame): DataFrame containing media data with a 'sales_volume' column.

    Returns:
        pd.DataFrame: DataFrame with processed sales volume data.
    """
    def convert_sales_volume_to_number(s):
        """
        Convert sales volume string to a number, handling ranges, specific markers, 
        and non-numeric characters.
        """
        try:
            if isinstance(s, str):
                s = s.replace('$', '').replace(',', '').replace('_', '').strip()
                # Handle cases like "Under $500,000" by taking the maximum or minimum value directly
                if 'under' in s.lower():
                    return float(s.split(' ')[1])
                elif 'over' in s.lower():
                    return float(s.split(' ')[1])  # Modify based on how you want to handle 'over' cases
                # Handle ranges by returning the lower bound
                return float(s.split(' ')[0].split('to')[0]) if 'to' in s else float(s)
        except ValueError:
            return None  # Return None for unconvertible values

    print("Initial count:", len(df_media))

    # Convert 'sales_volume' to numeric
    df_media['sales_volume'] = df_media['sales_volume'].apply(convert_sales_volume_to_number)

    # Drop rows where 'sales_volume' is NaN
    df_media_filtered = df_media.dropna(subset=['sales_volume'])

    # Filter rows where 'sales_volume' is below 1,000,000
    df_media_filtered = df_media_filtered[df_media_filtered['sales_volume'] < 1000000]

    print("Filtered count:", len(df_media_filtered))
    return df_media_filtered





In [4]:
df_tv_filtered = process_media_sales_volume(df_tv)
df_newspaper_filtered = process_media_sales_volume(df_newspaper)
df_radio_filtered = process_media_sales_volume(df_radio)

Initial count: 11487
Filtered count: 7246
Initial count: 41549
Filtered count: 32113
Initial count: 44117
Filtered count: 38822


In [12]:
def process_and_save_media_data(df_media, media_type):
    """
    Processes media DataFrame by counting occurrences for each state and county,
    then saves the results to a CSV file.

    Parameters:
        df_media (pd.DataFrame): DataFrame containing media data.
        media_type (str): String to identify the type of media data (e.g., 'tv', 'radio').

    Returns:
        None: The function saves the output directly to a CSV file.
    """
    # Group by 'Geography' and count occurrences
    county_counts = df_media.groupby(['Geography']).size().reset_index(name=f'count_{media_type}')

    # Construct the file path
    file_path = f'../artifacts/data/cleaned/{media_type}_county_counts.csv'

    # Save to CSV
    county_counts.to_csv(file_path, index=False)
    
    print(f"Data saved to {file_path}")


In [13]:
process_and_save_media_data(df_newspaper, 'newspaper')
process_and_save_media_data(df_tv, 'tv')
process_and_save_media_data(df_radio, 'radio')

Data saved to ../artifacts/data/cleaned/newspaper_county_counts.csv
Data saved to ../artifacts/data/cleaned/tv_county_counts.csv
Data saved to ../artifacts/data/cleaned/radio_county_counts.csv


In [16]:
df_tv_county_counts = pd.read_csv('../artifacts/data/cleaned/tv_county_counts.csv', on_bad_lines='skip')
df_radio_county_counts = pd.read_csv('../artifacts/data/cleaned/radio_county_counts.csv', on_bad_lines='skip')
df_newspaper_county_counts = pd.read_csv('../artifacts/data/cleaned/newspaper_county_counts.csv', on_bad_lines='skip')

In [19]:
import pandas as pd



# Merge the DataFrames on 'Geography', using full outer join to ensure all entries are included
merged_df = pd.merge(df_tv_county_counts, df_radio_county_counts, on='Geography', how='outer', suffixes=('_tv', '_radio'))
merged_df = pd.merge(merged_df, df_newspaper_county_counts, on='Geography', how='outer')

# If the column names need specific handling or renaming, adjust them accordingly:
merged_df.columns = ['Geography', 'count_tv', 'count_radio', 'count_newspaper']

# Replace NaN values with 0 for all count columns
final_merged_df = pd.merge(merged_df, df_shapefile_county, left_on='Geography', right_on='AFFGEOID', how='outer')

# Replace NaN values with 0 in relevant columns (assuming you only want to fill numerical columns)
columns_to_fill = ['count_tv', 'count_radio', 'count_newspaper']
for column in columns_to_fill:
    if column in final_merged_df.columns:
        final_merged_df[column] = final_merged_df[column].fillna(0).astype(int)

# Output the head of the final merged DataFrame to verify the results
print(final_merged_df.head())

# save to a CSV file 
final_merged_df.to_csv('../artifacts/data/cleaned/media_county_counts.csv', index=False)


        Geography  count_tv  count_radio  count_newspaper STATEFP COUNTYFP  \
0  0500000US01001         3            2                3      01      001   
1  0500000US01003         1           17               23      01      003   
2  0500000US01005         0            3                8      01      005   
3  0500000US01007         0            2                4      01      007   
4  0500000US01009         0            1                8      01      009   

   COUNTYNS        AFFGEOID  GEOID     NAME LSAD       ALAND      AWATER  \
0  00161526  0500000US01001  01001  Autauga   06  1539602123    25706961   
1  00161527  0500000US01003  01003  Baldwin   06  4117546676  1133055836   
2  00161528  0500000US01005  01005  Barbour   06  2292144655    50538698   
3  00161529  0500000US01007  01007     Bibb   06  1612167481     9602089   
4  00161530  0500000US01009  01009   Blount   06  1670103911    15015423   

                                            geometry  
0  POLYGON ((-86.92